In [2]:
import psycopg2

In [5]:
# Replace these values with your PostgreSQL connection details
dbname = 'chickflix'
user = 'ambukrehani'
password = 'ambuk'
host = 'localhost' # or your host

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [6]:
# Drop tables if they already exist
cursor.execute("DROP TABLE IF EXISTS Movies CASCADE;")
cursor.execute("DROP TABLE IF EXISTS Actors CASCADE;")
cursor.execute("DROP TABLE IF EXISTS Revenue CASCADE;")

# Create Movies table
cursor.execute("""
CREATE TABLE Movies (
    movie_id INTEGER PRIMARY KEY,
    title VARCHAR(255),
    release_date DATE
);
""")

# Create Actors table
cursor.execute("""
CREATE TABLE Actors (
    actor_id INTEGER PRIMARY KEY,
    name VARCHAR(255),
    movie_ids VARCHAR(255) -- This is a simplification for the example
);
""")

# Create Revenue table
cursor.execute("""
CREATE TABLE Revenue (
    revenue_id INTEGER PRIMARY KEY,
    movie_id INTEGER,
    amount DECIMAL,
    date DATE,
    FOREIGN KEY (movie_id) REFERENCES Movies (movie_id)
);
""")
#Create Characters table


conn.commit()


In [46]:
conn.rollback()

In [47]:
cursor.execute("""CREATE TABLE Characters (
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    movie_id INTEGER,
    actor_id INTEGER,
    FOREIGN KEY (movie_id) REFERENCES Movies(movie_id),
    FOREIGN KEY (actor_id) REFERENCES Actors(actor_id)
);
""")
conn.commit()

In [50]:
movies_data = [
    (1, 'Interstellar', '2014-11-07'),
    (2, 'Call Me by Your Name', '2017-11-24'),
    (3, 'Little Women', '2019-12-25'),
    (4, 'Dune', '2021-10-22'),
    (5, 'The French Dispatch', '2021-10-22')
]


actors_data = [
    (1, 'Timothée Chalamet', '2,3,4,5'),
    (2, 'Rebecca Ferguson', '4'),
    (3, 'Saoirse Ronan', '3'),
    (4, 'Zendaya', '4'),
    (5, 'Oscar Isaac', '4')
]


revenue_data = [
    (1, 1, 47, '2014-11-10'),
    (2, 2, 13, '2017-11-27'),
    (3, 3, 20, '2019-12-28'),
    (4, 4, 40, '2021-10-25'),
    (5, 5, 11, '2021-10-25')
]

characters_data = [
    ('Paul Atreides', 4, 1), 
('Lady Jessica', 4, 2),  
('Amy March', 3, 3),  
('Zach', 5, 1),
('Chani', 4, 4)
]

In [25]:
conn.rollback()

In [22]:
try:
    cursor.executemany("INSERT INTO Movies (movie_id, title, release_date) VALUES (%s, %s, %s)", movies_data)
except psycopg2.Error as e:
    print("Error:")
    print(e)

Error:
duplicate key value violates unique constraint "movies_pkey"
DETAIL:  Key (movie_id)=(1) already exists.



In [26]:
try:
    cursor.executemany("INSERT INTO Movies (movie_id, title, release_date) VALUES (%s, %s, %s)", movies_data)
except psycopg2.Error as e:
    print("Error:")
    print(e)

Error:
duplicate key value violates unique constraint "movies_pkey"
DETAIL:  Key (movie_id)=(1) already exists.



In [28]:
conn.rollback()

In [32]:
try:
    cursor.executemany("INSERT INTO Actors (actor_id, name, movie_ids) VALUES (%s, %s, %s)", actors_data)
except psycopg2.Error as e:
    print("Error:")
    print(e)

In [30]:
conn.rollback()

In [31]:
cursor.executemany("INSERT INTO Revenue (revenue_id, movie_id, amount, date) VALUES (%s, %s, %s, %s)", revenue_data)  # Assuming the first record is already inserted
conn.commit()


In [33]:
conn.commit()

In [52]:
conn.rollback()

In [53]:
try:
    cursor.executemany("INSERT INTO Characters (name, movie_id, actor_id) VALUES (%s, %s, %s)", characters_data)
except psycopg2.Error as e:
    print("Error:")
    print(e)


In [54]:
conn.commit()

In [40]:
conn.rollback()

In [ ]:
# Create dimension tables
cursor.execute("""
CREATE TABLE DimMovies (
    MovieID SERIAL PRIMARY KEY,
    Title VARCHAR(255),
    ReleaseDate DATE
)
""")

cursor.execute("""
CREATE TABLE DimActors (
    ActorID SERIAL PRIMARY KEY,
    Name VARCHAR(255)
)
""")

cursor.execute("""
CREATE TABLE DimDate (
    DateID SERIAL PRIMARY KEY,
    Date DATE,
    Year INT,
    Month INT,
    Day INT
)
""")

# Create fact table
cursor.execute("""
CREATE TABLE FactRevenue (
    RevenueID SERIAL PRIMARY KEY,
    MovieID INT,
    DateID INT,
    Revenue DECIMAL(10,2),
    FOREIGN KEY (MovieID) REFERENCES DimMovies(MovieID),
    FOREIGN KEY (DateID) REFERENCES DimDate(DateID)
)
""")



conn.commit()


In [77]:
conn.rollback()

In [59]:
cursor.execute("""
CREATE TABLE DimCharacters (
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    movie_id INTEGER,
    actor_id INTEGER,
    movie_title VARCHAR(255),
    actor_name VARCHAR(255),
    FOREIGN KEY (movie_id) REFERENCES DimMovies(movieid),
    FOREIGN KEY (actor_id) REFERENCES DimActors(actorid)
)""")
conn.commit()

In [41]:
cursor.execute("""
INSERT INTO DimMovies (movieid, title, releasedate)
SELECT movie_id, title, release_date FROM Movies;
""")
conn.commit()


In [42]:
cursor.execute("""
INSERT INTO DimActors (actorid, name)
SELECT actor_id, name FROM Actors;
""")
conn.commit()

In [60]:
cursor.execute("""
INSERT INTO DimCharacters (character_id, name, movie_id, actor_id, movie_title, actor_name)
SELECT c.character_id, c.name, c.movie_id, c.actor_id, m.title, a.name
FROM Characters c
JOIN Movies m ON c.movie_id = m.movie_id
JOIN Actors a ON c.actor_id = a.actor_id;

""")



In [61]:
conn.commit()

In [80]:
conn.rollback()

In [82]:
cursor.execute("""
INSERT INTO FactRevenue (revenueid, movieid, dateid, revenue)
SELECT r.revenue_id, r.movie_id, d.dateid, r.amount
FROM Revenue r
JOIN DimDate d ON r.date = d.date;
""")
conn.commit()


In [81]:
cursor.execute("""
INSERT INTO DimDate (date, year, month, day) VALUES
('2014-11-10', 2014, 11, 10),
('2017-11-27', 2017, 11, 27),
('2019-12-28', 2019, 12, 28),
('2021-10-25', 2021, 10, 25);
""")
conn.commit()


In [83]:
#Query to display the data which is has only Timothee Chalamet as an actor

cursor.execute("""
SELECT DISTINCT m.movieid, m.title
FROM DimMovies m
JOIN DimCharacters c ON m.movieid = c.movie_id
JOIN DimActors a ON c.actor_id = a.actorid
WHERE a.name = 'Timothée Chalamet';
""")



In [84]:
#movies with at least $10 million in revenue in their first 3 days

cursor.execute("""SELECT m.movieid, m.title
FROM DimMovies m
JOIN FactRevenue fr ON m.movieid = fr.movieid
WHERE fr.revenue >= 10;
""")

In [85]:
#movies that feature at least one character from "Dune" (excluding Timothée Chalamet's character),


cursor.execute("""SELECT DISTINCT m.movieid, m.title
FROM DimMovies m
JOIN DimCharacters c ON m.movieid = c.movie_id
JOIN DimActors a ON c.actor_id = a.actorid
WHERE m.title = 'Dune' AND a.name != 'Timothée Chalamet';
""")


